# ITER TBR Calculation
Goal : Build a working ITER TBR Simulation

Name : Husni Naufal Zuhdi (413821)

In [1]:
# Impor semua paket yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults, reset_defaults

import openmc
import paramak
import paramak_neutronics as nparamak
import neutronics_material_maker as nmm

Overwriting auto display for cadquery Workplane and Shape


## Step 1 : Define Materials

The materials use in this study are listed below
1. Plasma = Deuterium and Tritium
2. Blanket Fluid = LiF
3. Neutron Multiplier = Natural Pb
4. Moderator = Graphite
5. Reflector = Graphite
6. First Wall, Divertor, dan Vaccum Vessel= SS 316

In [2]:
# Plasma Material
plasma_mat = openmc.Material(name = 'Uranium 235', material_id=1)
plasma_mat.add_nuclide('U235', 1.0)
plasma_mat.set_density('g/cm3', 0.000001)
# plasma_mat = nmm.Material('DT_plasma') # Untuk plasma DT
# plasma_mat.openmc_material

# Blanket Fluid Material
enrichment_Li = 10
LiF_mat_nmm = nmm.Material.from_library(name='Lithium Fluoride',
                                        material_id=2,
                                        enrichment=enrichment_Li,
                                        enrichment_target='Li6',
                                        enrichment_type='ao'
                                       )
LiF_mat = LiF_mat_nmm.openmc_material

# Neutron Multiplier Material
neutron_multi_mat = openmc.Material(name = 'Lead', material_id=3)
neutron_multi_mat.add_element('Pb', 1)
neutron_multi_mat.set_density('g/cm3', 11.35)

# Moderator and Reflector Materials
mod_ref_mat = openmc.Material(name = 'Graphite', material_id=4)
mod_ref_mat.add_element('C', 1)
mod_ref_mat.set_density('g/cm3', 2.1)

# First Wall, Divertor and Vaccum Vessel Material
fw_mat_nmm = nmm.Material.from_library('Steel, Stainless 316', material_id=5)
fw_mat = fw_mat_nmm.openmc_material

/home/husninaufalz/miniconda3/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/husninaufalz/miniconda3/lib/python3.8/site-packages/openmc/mixin.py:68: IDWarning: Another Material instance already exists with id=5.
  warn(msg, IDWarning)


## Langkah 2 : Definisikan Geometri

In [3]:
# Color list : https://www.tug.org/pracjourn/2007-4/walden/color.pdf
# Make a ITER Tokamak Geometry Class
class ITERTokamak_mod(paramak.Reactor):
    """Create ITER geometry without TF Coils and PF Coils. This class modified
    from ITERTokamak function from paramak package"""
    
    def __init__(
        self,
        rotation_angle: float = 360.,
    ):
        super().__init__([])
        
        self.rotation_angle = rotation_angle
    
    def create_vessel_components(self) -> list:
        """Creates a 3d solids for each vessel component.

        Returns:
            A list of CadQuery solids: A list of 3D solid volumes
        """
        
        # Blanket reflector
        blanket_ref = paramak.BlanketFP(
            plasma=self.plasma,
#             thickness=3.65e2 - 3.5e2,
            thickness=15,
            start_angle=-70,
            stop_angle=230,
            rotation_angle=self.rotation_angle,
            vertical_displacement=self.plasma.vertical_displacement,
            offset_from_plasma=[[-70, 0, 90, 180, 230], [85, 55, 94, 51, 85]],
            name='blanket_ref',
            color=(0.9,0.4,0),
            stp_filename="blanket_ref.stp",
            stl_filename="blanket_ref.stl",
            material_tag='Graphite',
        )
        
        # Blanket Fluid
        blanket_fluid = paramak.BlanketFP(
            plasma=self.plasma,
#             thickness=3.95e2 - 3.65e2,
            thickness=30,
            start_angle=-70,
            stop_angle=230,
            rotation_angle=self.rotation_angle,
            vertical_displacement=self.plasma.vertical_displacement,
            offset_from_plasma=[[-70, 0, 90, 180, 230], [55, 25, 64, 21, 55]],
            name='blanket_fluid',
            color=(0.9,0.9,0),
            stp_filename="blanket_fluid.stp",
            stl_filename="blanket_fluid.stl",
            material_tag='Lithium Fluoride',
        )

        # Blanket first wall
        blanket_first_wall = paramak.BlanketFP(
            plasma=self.plasma,
#             thickness=4.0e2 - 3.95e2,
            thickness=5,
            start_angle=-70,
            stop_angle=230,
            rotation_angle=self.rotation_angle,
            vertical_displacement=self.plasma.vertical_displacement,
            offset_from_plasma=[[-70, 0, 90, 180, 230], [50, 20, 59, 16, 50]],
            name='blanket_first_wall',
            color=(0.6,0.7,1.0),
            stp_filename="blanket_first_wall.stp",
            stl_filename="blanket_first_wall.stl",
            material_tag='Steel, Stainless 316',
        )

        # SN Divertor
        divertor = paramak.ITERtypeDivertor(
            anchors=((4.34e2, -3.3e2), (5.56e2, -3.74e2)),
            coverages=(105, 125),
            lengths=(45, 75),
            radii=(68, 65),
            tilts=(-30, 2),
            dome_height=45,
            dome_pos=0.45,
            rotation_angle=self.rotation_angle,
            name='divertor',
            color=(0.3,0,0.6),
            stp_filename="divertor.stp",
            stl_filename="divertor.stl",
            material_tag='Steel, Stainless 316',
        )

        # Vacuum vessel
        divertor.points  # trigger the building of the points for divertor
        # the inner part of the vacuum vessel is computed from the outer
        # points of the blanket and the divertor
        vac_vessel_inner = paramak.RotateMixedShape(
            points=blanket_ref.outer_points + divertor.casing_points,
            rotation_angle=self.rotation_angle,
        )

        vac_vessel = paramak.RotateSplineShape(
            points=[
                (327.77, 36.5026668124882),
                (327.77, 73.37741270075162),
                (327.77, 108.31180820215741),
                (327.77, 143.2462037035632),
                (327.77, 178.18059920496898),
                (327.77, 213.11499470637477),
                (327.77, 248.04939020778068),
                (327.77, 282.98378570918646),
                (327.77, 317.9181812105922),
                (328.6121587814181, 368.23899806938385),
                (336.18303032328333, 422.4306297110355),
                (350.4835654579176, 457.5437492206628),
                (371.95910957013655, 492.47041663587777),
                (404.3208742000702, 522.0151685493631),
                (439.6516080621078, 544.4559826211985),
                (474.98234192414554, 556.3610266211815),
                (510.2245275810152, 564.0927634387052),
                (545.6438096482208, 565.1200145185009),
                (565.832800426528, 563.1864687746993),

                (580.9745435102584, 559.4390362932862),
                (616.3052773722961, 548.4109567158157),
                (651.6360112343338, 533.224020531035),
                (686.9667450963714, 515.3041214328789),
                (722.297478958409, 492.23516177329117),
                (757.6282128204466, 466.8689289401416),
                (792.9589466824843, 437.10619055069265),
                (825.7660566972336, 403.7167485984509),
                (853.525919017406, 369.42176700251196),
                (877.9209495411939, 333.90960594986575),
                (898.9511482685972, 300.5186330502012),
                (916.616515199616, 265.2383422522439),
                (932.5994662324425, 230.72194441870647),
                (946.0587934179808, 193.1122328856627),
                (956.1532888071343, 156.87835598377137),
                (962.8829523999035, 118.10702768634405),
                (967.9302000944803, 80.39197257542594),
                (968.7714080435763, 38.24754419835381),

                (968.7714080435763, 25.77097437642317),
                (964.5653682980957, -1.670738783514139),
                (956.9944967562304, -29.93883090626548),
                (956.1532888071343, -34.59540221679083),
                (946.0587934179808, -71.15339839027786),
                (931.7582582833464, -104.25874435511184),
                (914.9340993014238, -139.91477225259314),
                (898.9511482685972, -174.48160361826422),
                (883.8094051848669, -213.64300914878197),
                (867.8264541520404, -248.21908241802464),
                (851.0022951701176, -284.2078188440911),
                (834.1781361881949, -319.9470238737184),
                (818.1951851553683, -359.0978394110024),
                (800.5298182243495, -391.2313539579658),
                (776.1347877005617, -427.87174371008393),
                (744.1688856349085, -460.45530873911446),
                (708.8381517728709, -490.0255912806248),
                (673.5074179108332, -512.7040543014494),
                (638.1766840487956, -528.371873327094),
                (602.8459501867579, -539.0490644239661),
                (567.5152163247203, -546.1219131278361),
                (532.1844824626827, -548.9566889080664),
                (496.85374860064496, -547.7514325554811),
                (461.52301473860734, -541.3971156414638),
                (426.1922808765697, -527.596464992453),
                (390.8615470145321, -501.2796363633471),
                (360.57806084707124, -468.0473902249954),
                (340.389070068764, -431.4355817359209),
                (329.87397070506233, -399.072068113844),
                (327.770950832322, -357.4796824533661),
                (327.770950832322, -311.73270913617455),
                (327.770950832322, -276.79831363476876),
                (327.770950832322, -241.86391813336297),
                (327.770950832322, -206.92952263195718),
                (327.770950832322, -171.99512713055117),
                (327.770950832322, -137.06073162914538),
                (327.770950832322, -102.12633612773948),
                (327.770950832322, -67.19194062633369),

            ],
            cut=[vac_vessel_inner],  # to make a hollow shape
            rotation_angle=self.rotation_angle,
            name='vac_vessel',
            color=(0.9,0.9,0.9),
            stp_filename='vac_vessel.stp',
            stl_filename='vac_vessel.stl',
            material_tag='Steel, Stainless 316',
        )

        return [divertor, blanket_fluid, blanket_ref, blanket_first_wall, vac_vessel, vac_vessel_inner]
    
    
    def create_plasma(self) -> list:
        """Creates a 3d solids for the plasma.

        Returns:
            A list of CadQuery solids: A list of 3D solid volumes
        """

        self.plasma = paramak.Plasma(
            major_radius=6.2e2,
            minor_radius=2e2,
            elongation=1.7,
            triangularity=0.33,
            vertical_displacement=5.7e1,
            configuration="single-null",
            rotation_angle=self.rotation_angle,
            name='plasma',
            color=(0.7,0,0.7),
            stp_filename='plasma.stp',
            stl_filename='plasma.stl',
            material_tag='Uranium 235',
        )

        return [self.plasma]
    
    def create_solids(self):
        """Creates a 3d solids for each component.

        Returns:
            A list of CadQuery solids: A list of 3D solid volumes
        """

        plasma = self.create_plasma()
        vessel = self.create_vessel_components()

        shapes_and_components = plasma + vessel[:-1]
        self.shapes_and_components = shapes_and_components

        return shapes_and_components

In [4]:
# Test Geometry
my_reactor_test = ITERTokamak_mod()
my_reactor_test.show()

## Langkah 3 : Definisikan Sumber

In [5]:
from parametric_plasma_source import PlasmaSource, SOURCE_SAMPLING_PATH

my_plasma = PlasmaSource(
    ion_density_origin=1.09e20,
    ion_density_peaking_factor=1,
    ion_density_pedestal=1.09e20,
    ion_density_separatrix=3e19,
    ion_temperature_origin=45.9,
    ion_temperature_peaking_factor=8.06,
    ion_temperature_pedestal=6.09,
    ion_temperature_separatrix=0.1,
    elongation=1.7,
    triangularity=0.33,
    major_radius=6.2e2/100,
    minor_radius=2e2/100,
    pedestal_radius=(0.8 * 2e2) / 100,
    plasma_id=1,
    shafranov_shift=0.44789,
    ion_temperature_beta=6,
)

source = openmc.Source()
source.library = SOURCE_SAMPLING_PATH
source.parameters = str(my_plasma)

## Langkah 4 : Definisikan Tally Tritium Breeding Ration

In [ ]:
my_model = nparamak.NeutronicsModel(
    geometry=my_reactor_test,
    source=source,
    simulation_batches=10,  # this should be increased to get a better mesh tally result
    simulation_particles_per_batch=1000,  # this should be increased to get a better mesh tally result
    materials = {'plasma':plasma_mat,
                 'divertor':fw_mat,
                 'blanket_fluid':LiF_mat,
                 'blanket_ref':mod_ref_mat,
                 'blanket_first_wall':fw_mat,
                 'vac_vessel':fw_mat},
    mesh_tally_3d=['(n,Xt)','heating'],
    mesh_tally_2d=['(n,Xt)','heating'],
    cell_tallies=['(n,Xt)','heating', 'spectra'],
)

my_model.simulate()

In [ ]:
neutronics_model = paramak.NeutronicsModel(
    geometry=my_reactor,
    cell_tallies=['heating'],
    source=source,
    simulation_batches=10,
    simulation_particles_per_batch=10,
    materials={
        'pf_coil_mat': 'copper',
        'tf_coil_mat': 'copper',
        'blanket_mat': 'Li4SiO4',
    }
)

neutronics_model.simulate()

## Langkah 5 : Jalankan Perhitungan OpenMC